In [166]:
import random
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import IPython

In [167]:

def generate_trace_zero_constraints(dim, num_constraints):
    constraints = []
    for _ in range(num_constraints):
        M = np.random.randn(dim,dim)
        M = (np.transpose(M)+M)/2
        average = 0
        for i in range(dim):
            average += M[i,i]
        for i in range(dim):
            M[i,i] = M[i,i]-average/dim           
        constraints.append(M)
    x = np.random.randn(dim,1)

    percent_feasible = 0
    for i in range(len(constraints)):
        if np.matmul(np.transpose(x),np.matmul(constraints[i],x)) < 0:
            constraints[i] = -1*constraints[i]
        elif np.matmul(np.transpose(x),np.matmul(constraints[i],x)) ==0:
            print('Oh no! Something evaluates to 0!')
    
    return constraints, percent_feasible

In [163]:
def find_feasible_point(dim, num_constraints, scale, batch_size, initial_step, decay_rate):
    #get matrices
    constraints, percent_feasible = generate_trace_zero_constraints(dim, num_constraints)     
    
    #learned feasible point
    X = tf.Variable(tf.random_normal([dim, 1], 0., scale))
    stacked_matrices = tf.placeholder(tf.float32, [None, dim])
    exact_batch_size = tf.placeholder(tf.int32)

    #tf.reduce sum is for inner product x & Px
    
    quadratic = tf.reduce_sum(tf.reshape(tf.multiply(tf.transpose(tf.tile(tf.transpose(X), [1,exact_batch_size])), tf.matmul(stacked_matrices, X)),[exact_batch_size,dim]),axis=1)
    
    #loss = tf.reduce_mean(tf.maximum(0., 1.-quadratic))#does coordinatewise?
    loss = tf.reduce_mean(tf.log(1+tf.exp(-1*quadratic)))
    sess = tf.InteractiveSession()

    global_step = tf.Variable(1.0, trainable=False)
    learning_rate = tf.train.exponential_decay(initial_step, global_step, num_constraints, decay_rate)
    #learning_rate = tf.train.inverse_time_decay(initial_step, global_step, num_constraints, decay_rate, staircase=True)
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)
    var_grad = tf.gradients(loss,X)[0]   
    correct = tf.less(tf.zeros([1,exact_batch_size]),quadratic)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    #for shuffling
    indices = np.random.permutation(num_constraints)
    
    tf.global_variables_initializer().run() 
    iteration = 0

    initials = sess.run([accuracy,loss],
                          feed_dict={stacked_matrices: np.concatenate([constraints[i] for i in range(num_constraints)]),
                                    exact_batch_size:num_constraints})
    old_loss = []
    old_accuracy = []
    old_loss.append(initials[1])
    old_accuracy.append(initials[0])
    for _ in range(8000):
        #print('iter {}'.format(_))
        x = sess.run([accuracy, loss, var_grad, learning_rate,X],
                              feed_dict={stacked_matrices: np.concatenate([constraints[i] for i in range(num_constraints)]),
                                        exact_batch_size:num_constraints})
        #print('accuracy', x[0], 'loss', x[1], 'gradient', np.linalg.norm(x[2],ord='fro'), 'learning_rate', x[3])
        
        iteration += 1
        IPython.display.clear_output(True)
        plt.plot(np.log(old_loss))
        angle = np.matmul(x[4].T,x[2])/(np.linalg.norm(x[4])*np.linalg.norm(x[2]))
        plt.title('accuracy:{:2f} loss:{:2f} grad:{:2f} step:{:2f} norm:{:2f} angle:{:2f}'.format(x[0], x[1], np.linalg.norm(x[2],ord='fro'), x[3], np.linalg.norm(x[4]), angle[0][0]))
        plt.show()
        curr_x = x[4]
        gg = (np.dot(sum([-2/(1+np.exp(curr_x.T@Pi@curr_x))*Pi for Pi in constraints]),curr_x)/num_constraints)
        #gg = gg/np.linalg.norm(gg)
        print([gg])#, x[4]/np.linalg.norm(x[4])])
        #print(x[4])
        print(percent_feasible)
        
        old_loss.append(x[1])
        old_accuracy.append(x[0])

#         if x[0] == 1 or x[1]==0 :
#             break


        for i in range(1, int(np.round(num_constraints/batch_size))):
            if len(constraints[batch_size*(i-1): batch_size*i]) > 0:
                sess.run([train_step],
                          feed_dict={stacked_matrices: np.concatenate([constraints[j] for j in indices[batch_size*(i-1): batch_size*i]]),
                                    exact_batch_size: batch_size})
            else:
                sess.run([train_step],
                          feed_dict={stacked_matrices: np.concatenate([constraints[j] for j in indices[batch_size*(i-1):]]),
                                    exact_batch_size: len(indicies[batch_size*(i-1):])})
        indices = np.random.permutation(num_constraints)
        #shuffle samples        
    x = sess.run([X,correct],
                          feed_dict={stacked_matrices: np.concatenate([constraints[i] for i in range(num_constraints)]),
                                    exact_batch_size:num_constraints})

    return old_loss, old_accuracy, iteration

In [168]:
#find_feasible_point          (dim, num_constraints, scale, batch_size, initial_step, decay_rate)
t = time.time()
for _ in range(1):
    [matrices, point] = find_feasible_point(5, 100, 1e-3, 10,.001,.99)
print(time.time()-t)

KeyboardInterrupt: 

In [ ]:
num_exp = 8001
average_loss =np.zeros(num_exp)
average_accuracy= np.zeros(num_exp)
n = 20.0
std_dev_loss = np.zeros(num_exp)
std_dev_accuracy = np.zeros(num_exp)

for loss in losses:
    for i in range(num_exp):
        if i < len(loss):
            average_loss[i] += np.log(loss[i])
        else:
            average_loss[i]+= np.log(loss[len(loss)-1])

average_loss = average_loss / n

for loss in losses:
    for i in range(num_exp):
        if i < len(loss):
            std_dev = (np.log(loss[i])-average_loss[i])*(np.log(loss[i])-average_loss[i])
            std_dev_loss[i]+=std_dev
        else:
            std_dev = (np.log(loss[len(loss)-1])-average_loss[i])*(np.log(loss[len(loss)-1])-average_loss[i])
            std_dev_loss[i]+=std_dev


for accuracy in accuracies:
    for i in range(num_exp):
        if i < len(accuracy):
            average_accuracy[i] += accuracy[i]
        else:
            average_accuracy[i] += accuracy[len(accuracy)-1]

average_accuracy = average_accuracy / n

for accuracy in accuracies:
    for i in range(len(accuracy)):
        std_dev= (accuracy[i]-average_accuracy[i])*(accuracy[i]-average_accuracy[i])
        std_dev_accuracy[i]+=std_dev
    for i in range(num_exp-len(accuracy)):
        std_dev = (accuracy[len(accuracy)-1]-average_accuracy[len(accuracy)+i])*(accuracy[len(accuracy)-1]-average_accuracy[len(accuracy)+i])
        std_dev_accuracy[i+len(accuracy)]+=std_dev

for i in range(len(std_dev_loss)):
    std_dev_loss[i] = np.sqrt(std_dev_loss[i]/n)
    std_dev_accuracy[i] = np.sqrt(std_dev_accuracy[i]/n)
    
#plt.errorbar([i for i in range(len(average_accuracy[50:1431]))],average_accuracy[50:1431], yerr=std_dev_accuracy[50:1431], fmt='o')
lower = 50
step = 50
upper = 700
x = [i for i in range(len(average_accuracy))]
plt.plot(x[lower:upper] , average_accuracy[lower:upper])
plt.errorbar(x[lower:upper:step], average_accuracy[lower:upper:step], yerr=std_dev_accuracy[lower:upper:step],linestyle = 'None')


#plt.plot(average_accuracy[100:527])
plt.show()